# Face Verification

The basic idea behind face recognition can be illustrated using the example below where I employ the use of the following: 
- Transfer learning: We use the VGG 16 model and the VGG Face model. Freeze layers along with modifying the neural network architecture
- Use distance functions (euclidean and cosine) to determine matches

In [57]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential, load_model, model_from_json, Model
from keras.layers import Activation , Dropout
from keras.layers.core import Dense, Flatten
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from PIL import Image

## Transfer Learning 

Lets load in the VGG16 pretrained model. Here we do the following: 
- (i) Load in the model
- (ii) Modify architecture: remove last 2 dense layers. Add softmax activation on the resultant last layer
- (iii) Freeze VGG_16 face model weights

### (i) Loading in the model

In [3]:
"""
KERAS.APPLICATIONS
- Keras has a repo of pretrained models that you can simply pull from to further your training or use it however you want
"""
vgg16_model = keras.applications.vgg16.VGG16(weights='imagenet')

In [4]:
# Lets check out the pre-trained model we're going to tune!
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

### (ii) Modify model architecture
- remove last 3 dense layers. 
- Add softmax activation
- freezing our weights

In [30]:
# Transform here
face_model = Sequential()

#creating our own modelfrom the VGG 16
for layer in vgg16_model.layers[:-2]:
    face_model.add(layer)

In [31]:
#Adding softmax activation
face_model.add(Activation('softmax'))

In [33]:
#lets view our new model
face_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

### (iii) Freezing all the weights!

The VGG16 model has already been pretrained on a large dataset and therefore we simply "transfer" the use of the model for our purpose as the convolutional neural network already has the capability of succeafully identifying image features

In [34]:
#freezing all weights
for layer in face_model.layers:
    layer.trainable = False

## Using the model

We will do the following here: 
- (i) Read in our images and resize
- (ii) represent the flattened images 
- (iii) then performing face verification

### (i) Reading & resizing

In [8]:
#read in images and resize
def resize(my_image):
    img = Image.open(my_image)
    img.load()
    img = img.resize((224, 224), Image.ANTIALIAS)    
    data = np.asarray( img, dtype="int32" )
    return data.reshape(1,224,224,3)             #model expects 4 dims
img1 = resize('C:/Users/Darshil/Desktop/and_darsh.jpg')
img2 = resize('C:/Users/Darshil/Desktop/darsh.jpg')

### (ii) Lets represent an image as a flattened layer

In [35]:
# Here we have simply passed the 2 images through the custom VGG16 convolutional neural network we've modified 
#model.predict(img1)
face_model.predict(img2)

array([[6.4339283e-16, 6.4339283e-16, 3.6733999e-14, ..., 6.4339283e-16,
        3.7570569e-10, 1.2395185e-13]], dtype=float32)

In [68]:
#define thresholds 
#epsilon = 0.40 #cosine similarity
epsilon = 1 #euclidean distance

def euclidean_distance(image1,image2, pickmodel):
    # Lastly we can perform the face verification
    p_img1 = pickmodel.predict(image1)
    p_img2 = pickmodel.predict(image2)

    #distance function
    diff = p_img1 - p_img2
    a = np.sum(np.multiply(diff,diff))
    euc_dist = np.sqrt(a) 
        
    if euc_dist < 1 and euc_dist > 0.5:
        print ('Possible match!')    
    elif euc_dist < 0.5:
        print ('more confident match!')
    else:
        print ('Not a good match')
    
    print (euc_dist, '\n')       
    return euc_dist

euclidean_distance(resize('leo1.jpg'),resize('darsh1.jpg'),face_model)
euclidean_distance(resize('leo1.jpg'),resize('leo2.jpg'),face_model)
euclidean_distance(resize('leo2.jpg'),resize('leo3.jpg'),face_model)

Not a good match
1.0354294 

Possible match!
0.86949027 

Possible match!
0.916277 



0.916277

In [55]:
distance(resize('C:/Users/Darshil/Desktop/darsh1.jpg'),resize('C:/Users/Darshil/Desktop/randface1.jpg'))

Possible match!
0.8522137 



0.8522137

# Using the VGG Face model

Although the architecture is kind of similar, the VGG face model was trained on an entirely different dataset and hence has different weights. 

We will do the following: 
- (i) create a VGG FACE MODEL 
- (ii) Load up the weights
- (iii) Implement

### (i)

In [58]:
# Model
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [59]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_14 (ZeroPaddi (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_15 (ZeroPaddi (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_16 (ZeroPaddi (None, 114, 114, 64)      0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 112, 112, 128)     73856     
__________

In [60]:
#here we load the weights
model.load_weights('vgg_face_weights.h5', by_name=True)

In [61]:
# Here we have simply passed the 2 images through the custom VGG16 FACE NETWORK
model.predict(img2)

array([[0.00042571, 0.00037262, 0.00037893, ..., 0.00038766, 0.00037345,
        0.00040366]], dtype=float32)

In [74]:
#now we implement and find distances!
euclidean_distance(resize('leo1.jpg'),resize('darsh1.jpg'),model)
euclidean_distance(resize('leo1.jpg'),resize('leo2.jpg'),model)
euclidean_distance(resize('leo2.jpg'),resize('leo3.jpg'),model)

more confident match!
0.00062461285 

more confident match!
0.00020716371 

more confident match!
0.0004658442 



0.0004658442

In [73]:
euclidean_distance(resize('leo1.jpg'),resize('me3.jpg'),model)
#PENDING, CHANGE THRESHOLD FOR VGG FACE MODEL

more confident match!
0.00043190693 



0.00043190693

### Using an intermediary layer which doesn't seem 

In [65]:
vgg_face_descriptor = Model(inputs=model.layers[0].input
, outputs=model.layers[-2].output)

In [66]:
euclidean_distance(resize('leo1.jpg'),resize('darsh1.jpg'),vgg_face_descriptor)
euclidean_distance(resize('leo1.jpg'),resize('leo2.jpg'),vgg_face_descriptor)
euclidean_distance(resize('leo2.jpg'),resize('leo3.jpg'),vgg_face_descriptor)

array([[ 0.1132349 , -0.01994853, -0.00317083, ...,  0.01962105,
        -0.01773274,  0.06005107]], dtype=float32)